<a href="https://colab.research.google.com/github/vieirafrancisco/machine-learning-ufal/blob/master/arvores_de_decisao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive

drive.mount('drive')
os.chdir('drive/My Drive/UFAL/machine-learning-ufal/video-game-sales')

Mounted at drive


# Sobre o dataset
**AVISO**: O motivo pelo qual mudei o dataset foi pelo fato do anterior ter limitações para fazer classificação, pela quantidade de dados e atributos não descritores. Porém foi importante para extrair algumas análises interessantes.

**Synthetic Financial Datasets For Fraud Detection**

[kaggle link](https://www.kaggle.com/ntnu-testimon/paysim1)

colunas:  
- step - unidade de tempo referente a uma hora (até 744 - 31 dias).

- type - CASH-IN (deposito), CASH-OUT (saque), DEBIT (deito), PAYMENT (pagamento) and TRANSFER (transferência).

- amount - valor da transação

- nameOrig - cliente que começou a transação

- oldbalanceOrg - balanço inicial

- newbalanceOrig - balanço após transação

- nameDest - cliente que recebeu a transação

- oldbalanceDest - balanço inicial do cliente que recebeu a transação

- newbalanceDest - balanço após transação do cliente que recebeu a transação

- isFraud - 1 para faude e 0 não fraude

- isFlaggedFraud - flag para tranferências massivas acima de 200.000

# Imports

In [2]:
from collections import Counter
import numpy as np
import pandas as pd

# Load Data

In [3]:
# load data
df = pd.read_csv('data/fraud_dataset.csv', index_col=False)

# Análise preliminar

In [4]:
# análise preliminar

# verificar quantidade de destinatários em transações fraudulentas
fraud_transactions = df[df['isFraud']==1]
print(f"quantidade de transações fraudulentas: {len(fraud_transactions)}")
print(f"quantidade de destinatários em transações fraudulentas: {len(set(fraud_transactions['nameDest']))}")

# verificar quantidade de destinatários em transações não fraudulentas
non_fraud_transactions = df[df['isFraud']==0]
print(f"quantidade de transações fraudulentas: {len(non_fraud_transactions)}")
print(f"quantidade de destinatários em transações fraudulentas: {len(set(non_fraud_transactions['nameDest']))}")

quantidade de transações fraudulentas: 8213
quantidade de destinatários em transações fraudulentas: 8169
quantidade de transações fraudulentas: 6354407
quantidade de destinatários em transações fraudulentas: 2719685


In [5]:
# vericar letra que fica no inicio de cada id (M vem de Merchant - Comerciante)
print(f"origem: {Counter(list(map(lambda x: x[0], df['nameOrig'])))}") # como no de origem é constante então vou remover esse atributo
print(f"destinatário: {Counter(list(map(lambda x: x[0], df['nameDest'])))}") # esse atributo vai ser substituido pela primeira letra

origem: Counter({'C': 6362620})
destinatário: Counter({'C': 4211125, 'M': 2151495})


In [6]:
# observações:
# 1 - Os nomes (ids) parecem não influênciar diretamente em transações fraudulentas
# 2 - A quantidade de tuplas com transações fraudulentas é bem discrepante com relação as tuplas que não são.
# Será necessário um balanceamento antes de fazer o processo de classificação.

# Alguns pré-processamentos

In [7]:
# pre-processamento

# remover tuplas que não serão usadas
pre_df = df.drop(['nameOrig', 'step', 'isFlaggedFraud'], axis=1)

In [8]:
# substituir ids dos destinatários por somente o primero caractere
pre_df['nameDest'] = list(map(lambda x: x[0], pre_df['nameDest']))

In [9]:
# discretizar atributos categóricos
def get_indices(values):
    unique_values = list(values.unique())
    indices = {value: unique_values.index(value) for value in unique_values}
    return indices

def discretize(df):
    tmp_df = df.copy()
    categorical = [column for column in tmp_df.columns if type(tmp_df[column].iloc[0]) == str]
    for column in categorical:
        indices = get_indices(tmp_df[column])
        tmp_df[column] = list(map(lambda x: indices[x],tmp_df[column]))
    return tmp_df
    
pre_df = discretize(pre_df)

In [10]:
# fazer re-escala dos atributos
# (x - min)/(max - min)
for column in pre_df.columns:
    values = pre_df[column]
    _max = max(values)
    _min = min(values)
    pre_df[column] = list(map(lambda x: (x-_min)/(_max-_min), values))

In [11]:
pre_df.head()

,type,amount,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,0.00,0.000106,0.002855,0.003233,0.0,0.000000,0.0,0.0
1,0.00,0.000020,0.000357,0.000391,0.0,0.000000,0.0,0.0
2,0.25,0.000002,0.000003,0.000000,1.0,0.000000,0.0,1.0
3,0.50,0.000002,0.000003,0.000000,1.0,0.000059,0.0,1.0
4,0.00,0.000126,0.000697,0.000603,0.0,0.000000,0.0,0.0


# Classificação

In [12]:
# classificação com árvores de decisão

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, KFold

In [13]:
# hold-out com dataset desbalanceado
y = pre_df['isFraud']
X = pre_df.drop(['isFraud'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 

In [14]:
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)
accuracy_score(y_test, dt_clf.predict(X_test))

0.9997304569501243

In [15]:
#rf_clf = RandomForestClassifier()
#rf_clf.fit(X_train, y_train)
#accuracy_score(y_test, rf_clf.predict(X_test))

In [16]:
# balancear dataset
is_fraud = pre_df[pre_df['isFraud']==1]
non_is_fraud = pre_df[pre_df['isFraud']==0][9000:18000]
b_df = is_fraud.append(non_is_fraud)

In [17]:
y = b_df['isFraud']
X = b_df.drop(['isFraud'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 

In [18]:
# função de classificação
# retorna a accuracy, precision, recall
def classify(clf, X, y, cv=None):
    if cv is not None:
        accuracy, precision, recall = cross_validation(clf, X, y, cv)
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        accuracy, precision, recall = hold_out(clf, X_train, X_test, y_train, y_test)
    return {"accuracy": accuracy, "precision": precision, "recall": recall}
    

def hold_out(clf, X_train, X_test, y_train, y_test):
    # classificar
    clf.fit(X_train, y_train)
    # predição
    y_predict = clf.predict(X_test)

    # métricas
    acuracia = accuracy_score(y_test, y_predict)
    precision = precision_score(y_test, y_predict)
    recall = recall_score(y_test, y_predict)
    return (acuracia, precision, recall)

def cross_validation(clf, X, y, cv):
    kf = KFold(n_splits=cv, shuffle=True, random_state=42)
    metrics = {"accuracy": [], "precision": [], "recall": []}
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        accuracy, precision, recall = hold_out(clf, X_train, X_test, y_train, y_test)
        metrics["accuracy"].append(accuracy)
        metrics["precision"].append(precision)
        metrics["recall"].append(recall)
    return (np.mean(metrics["accuracy"]), np.mean(metrics["precision"]), np.mean(metrics["recall"]))

In [19]:
classify(DecisionTreeClassifier(), X, y)

{'accuracy': 0.991867557362765,
 'precision': 0.9939061547836685,
 'recall': 0.989084293511219}

In [20]:
classify(RandomForestClassifier(), X, y)

{'accuracy': 0.9939006680220738,
 'precision': 0.991166077738516,
 'recall': 0.9964476021314387}

In [21]:
# classicação com validação cruzada
classify(DecisionTreeClassifier(), X, y, cv=10)

{'accuracy': 0.9931446684766507,
 'precision': 0.9915987572100308,
 'recall': 0.9940684288663999}

In [22]:
classify(RandomForestClassifier(), X, y, cv=10)

{'accuracy': 0.9952362731064847,
 'precision': 0.9941710715877352,
 'recall': 0.9959042913834038}

In [23]:
# conclusões:
# em geral decision tree e random forest conseguiram discriminar o problema com o dataset apresentado
# mesmo no modo hold-out o modelo fez boas previsões com os atributos padrões dos classificadores.

In [24]:
# salvar dataset pre-processado
pre_df.to_csv('data/fraud_dataset_pre_processed.csv')